In [ ]:
!pip install ffsim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 156.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 123.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.9 MB/s eta 0:00:00


In [ ]:
import pyscf
import pyscf.mcscf
import ffsim
import numpy as np
import scipy.optimize

#mol = pyscf.gto.Mole()

##### Build an H10 molecule with bond length 2 Å
# bond_length = 1.0

# mol.build(
#     atom=[["H", (0, 0, i * bond_length)] for i in range(12)],
#     basis="sto-6g"
# )

bond_length = 1.2

mol = pyscf.gto.Mole()
mol.build(
    atom=[
        ["N", (0, 0, 0)],
        ["N", (0, 0,  bond_length)]
    ],
    basis="sto-6g",
    unit="Angstrom"
)


n_active_electrons = mol.nelectron
n_active_orbitals = mol.nao_nr()

active_space = range(n_active_orbitals)

# Get molecular data and molecular Hamiltonian (one- and two-body tensors)
scf = pyscf.scf.RHF(mol).run()
mol_data = ffsim.MolecularData.from_scf(scf, active_space=active_space)
norb = mol_data.norb
nelec = mol_data.nelec
mol_hamiltonian = mol_data.hamiltonian

# Compute FCI energy
mol_data.run_fci()

print(f"norb = {norb}")
print(f"nelec = {nelec}")

converged SCF energy = -108.535614528761
Parsing /tmp/tmpvuiim95m
converged SCF energy = -107.938194421902


Overwritten attributes  get_ovlp get_hcore  of <class 'pyscf.scf.hf_symm.SymAdaptedRHF'>
/usr/local/lib/python3.12/dist-packages/pyscf/gto/mole.py:1300: UserWarning: Function mol.dumps drops attribute energy_nuc because it is not JSON-serializable
  warnings.warn(msg)
/usr/local/lib/python3.12/dist-packages/pyscf/gto/mole.py:1300: UserWarning: Function mol.dumps drops attribute intor_symmetric because it is not JSON-serializable
  warnings.warn(msg)


CASCI E = -108.727113223000  E(CI) = -130.335182668900  S^2 = 0.0000000
norb = 10
nelec = (7, 7)


CO result, at bond length 1.5 A, -112.354719993162 vs  -112.34728088857229 (full,840 paras) and -112.33880911048277 (local,476)

In [ ]:
import numpy as np
from pyscf import cc

# Get CCSD t2 amplitudes for initializing the ansatz
ccsd = cc.CCSD(
    scf, frozen=[i for i in range(mol.nao_nr()) if i not in active_space]
).run()

# Construct UCJ operator
n_reps = 3
operator = ffsim.UCJOpSpinBalanced.from_t_amplitudes(ccsd.t2, n_reps=n_reps)

# Construct the Hartree-Fock state to use as the reference state
reference_state = ffsim.hartree_fock_state(norb, nelec)

# Apply the operator to the reference state
ansatz_state = ffsim.apply_unitary(reference_state, operator, norb=norb, nelec=nelec)

# Compute the energy ⟨ψ|H|ψ⟩ of the ansatz state
hamiltonian = ffsim.linear_operator(mol_hamiltonian, norb=norb, nelec=nelec)
energy = np.real(np.vdot(ansatz_state, hamiltonian @ ansatz_state))
print(f"Energy at initialization: {energy}")
print('number of paras', len(operator.to_parameters()))

E(CCSD) = -108.7210654096777  E_corr = -0.1854508809166108
Energy at initialization: -108.68327816921266
number of paras 630


In [ ]:
import scipy.optimize

def fun(x):
    # Initialize the ansatz operator from the parameter vector
    operator = ffsim.UCJOpSpinBalanced.from_parameters(x, norb=norb, n_reps=n_reps)
    # Apply the ansatz operator to the reference state
    final_state = ffsim.apply_unitary(reference_state, operator, norb=norb, nelec=nelec)
    # Return the energy ⟨ψ|H|ψ⟩ of the ansatz state
    return np.real(np.vdot(final_state, hamiltonian @ final_state))


result = scipy.optimize.minimize(
    fun, x0=operator.to_parameters(), method="L-BFGS-B", options=dict(maxiter=100000, maxfun=5000000, ftol=1e-7)
)

print(f"Number of parameters: {len(result.x)}")
print(result)

Number of parameters: 630
  message: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
  success: True
   status: 0
      fun: -108.70464877225912
        x: [-3.160e-01  3.167e-01 ...  9.639e-03 -3.909e-02]
      nit: 28
      jac: [-2.004e-04 -1.390e-03 ... -3.666e-04 -3.055e-04]
     nfev: 20192
     njev: 32
 hess_inv: <630x630 LbfgsInvHessProduct with dtype=float64>


In [ ]:
pairs_aa = [(p, p + 1) for p in range(norb - 1)]
pairs_ab = [(p, p) for p in range(norb)]
interaction_pairs = (pairs_aa, pairs_ab)


def fun(x):
    operator = ffsim.UCJOpSpinBalanced.from_parameters(
        x, norb=norb, n_reps=n_reps, interaction_pairs=interaction_pairs
    )
    final_state = ffsim.apply_unitary(reference_state, operator, norb=norb, nelec=nelec)
    return np.real(np.vdot(final_state, hamiltonian @ final_state))


result = scipy.optimize.minimize(
    fun,
    x0=operator.to_parameters(interaction_pairs=interaction_pairs),
    method="L-BFGS-B",
    options=dict(maxiter=100000, maxfun=5000000, ftol=1e-7),
)
print(f"Number of parameters: {len(result.x)}")
print(result)

Number of parameters: 357
  message: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
  success: True
   status: 0
      fun: -108.71526456778797
        x: [-4.503e-01  3.015e-01 ... -2.255e-01 -1.857e-01]
      nit: 143
      jac: [ 2.103e-04  3.070e-04 ...  0.000e+00 -1.148e-03]
     nfev: 53700
     njev: 150
 hess_inv: <357x357 LbfgsInvHessProduct with dtype=float64>


In [ ]:
from collections import defaultdict

from ffsim.optimize import minimize_linear_method


# Define function that converts a list of parameters to the corresponding state vector
def params_to_vec(x: np.ndarray) -> np.ndarray:
    operator = ffsim.UCJOpSpinBalanced.from_parameters(
        x, norb=norb, n_reps=n_reps, interaction_pairs=interaction_pairs
    )
    return ffsim.apply_unitary(reference_state, operator, norb=norb, nelec=nelec)


# Define a callback function used to save optimization information (this is optional)
info = defaultdict(list)


def callback(intermediate_result: scipy.optimize.OptimizeResult):
    # The callback function is called after each iteration. It accepts
    # an OptimizeResult object storing the parameters and function value at
    # the current iteration, and possibly other information
    info["x"].append(intermediate_result.x)
    info["fun"].append(intermediate_result.fun)
    if hasattr(intermediate_result, "jac"):
        info["jac"].append(intermediate_result.jac)
    if hasattr(intermediate_result, "regularization"):
        info["regularization"].append(intermediate_result.regularization)
    if hasattr(intermediate_result, "variation"):
        info["variation"].append(intermediate_result.variation)


# Optimize with the linear method
result = minimize_linear_method(
    params_to_vec,
    hamiltonian,
    x0=operator.to_parameters(interaction_pairs=interaction_pairs),
    maxiter=1000,
    callback=callback,
)

# Print some information
print(f"Number of parameters: {len(result.x)}")
print(result)
print()
for i, (fun, jac, regularization, variation) in enumerate(
    zip(info["fun"], info["jac"], info["regularization"], info["variation"])
):
    print(f"Iteration {i + 1}")
    print(f"    Energy: {fun}")
    print(f"    Norm of gradient: {np.linalg.norm(jac)}")
    print(f"    Regularization hyperparameter: {np.linalg.norm(regularization)}")
    print(f"    Variation hyperparameter: {np.linalg.norm(variation)}")

Number of parameters: 357
 message: Convergence: Relative reduction of objective function <= ftol.
 success: True
     fun: -108.71927702442633
       x: [-1.231e-01  1.996e-01 ... -3.084e-01 -2.942e-01]
     nit: 92
     jac: [ 5.547e-04 -1.567e-04 ... -6.399e-07  1.519e-06]
    nfev: 69604
    njev: 93
  nlinop: 36403

Iteration 1
    Energy: -108.67194400801817
    Norm of gradient: 0.0941630448968623
    Regularization hyperparameter: 0.0018318211015259924
    Variation hyperparameter: 0.9995350890512504
Iteration 2
    Energy: -108.68352023577549
    Norm of gradient: 0.05363166178576294
    Regularization hyperparameter: 0.0030409191456738107
    Variation hyperparameter: 0.9995353033152398
Iteration 3
    Energy: -108.69011190884964
    Norm of gradient: 0.04626882820215872
    Regularization hyperparameter: 0.005193473572002985
    Variation hyperparameter: 0.9995352349289552
Iteration 4
    Energy: -108.69722128813942
    Norm of gradient: 0.045712980241453025
    Regularizati